In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import udf, col, sum, avg, count, max, when, desc, asc, lit
from pyspark.sql.types import StringType, DoubleType, FloatType

import os
import numpy as np
import pandas as pd
import tables
import time

In [2]:
spark = SparkSession.builder \
    .appName("country_hotness") \
    .config("spark.master", "yarn") \
    .config("spark.submit.deployMode", "client") \
    .config("spark.executor.cores", 1)\
    .config("spark.cores.max", 1)\
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)


In [3]:
start_time = time.time()

In [4]:
df = sqlContext.read.csv("hdfs://Group10-master:9000/user/hadoop/millionsongsubsetinflated.csv",
                        header='true', inferSchema='true')

df.show(vertical = True)

In [5]:
df_filtered = df.filter("Country is not null").filter("Hotttness > 0")

df_filtered.show(1, vertical = True)
df_filtered.count()

In [6]:


df_grouped = df_filtered.groupBy("Country")

In [7]:
df_top_sum = df_grouped.agg(sum("Hotttness").alias("total_hotness"), count("Country").alias("count"))

df_top_sum.sort(desc("total_hotness")).show()

In [8]:
df_grouped_avg = df_grouped.agg(avg("Hotttness").alias("average_hotness"), count("Country").alias("count"))

df_grouped_avg.sort(desc("average_hotness")).show()

In [9]:
df_grouped_filtered = df_grouped_avg.filter("`count` >= 5")

df_grouped_filtered.sort(desc("average_hotness")).show()

In [10]:
df_grouped_avg.sort(asc("average_hotness")).show()

In [11]:
df_grouped_filtered.sort(asc("average_hotness")).show()

In [12]:
# Check that total count is correct
df_grouped_avg.groupBy().sum().collect()

In [13]:
print("Time taken: ", time.time() - start_time)

In [14]:
sc.stop()